# Experiment design

Comparison of the Joshi-Boyd rounding heuristic and a brute force approach that searches through all possible designs

In [69]:
import numpy as np
from itertools import combinations
from numpy.linalg import inv, svd
import pandas as pd
import matplotlib.pyplot as plt
import cvxopt 
import cvxpy as cp


In [70]:
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCS']


In [71]:
# select the random (m x n) orthogonal matrix, U, that we will use
np.random.seed(1)
c_means = pd.read_csv('../../data/col_means.csv')
assoc_mat = np.array(pd.read_csv('../../data/uw_58_ratings_matrix.csv'))[:,1:]
assoc_mat = assoc_mat- c_means.V1.values
assoc_mat = assoc_mat.T
assoc_mat = assoc_mat.astype('float')



m = 58  # number of rows
n = 30   # number of columns
#U,S,Vh = svd(np.random.randn(m,n), full_matrices=False)
U,S,Vh = svd(assoc_mat, full_matrices=False)


In [72]:
# evaluate cost function, picking subset "select" from rows of U
def costfun(U,select):
    m,n = U.shape
    return 1/n * inv(U[select,:].T @ U[select,:]).trace()

In [73]:
# Use convex relaxation with rounding
# e.g. as in Joshi-Boyd paper:  doi: 10.1109/TSP.2008.2007095
def BoydHeuristic(U,r):
    X = cp.Variable((n,n), symmetric=True)
    v = cp.Variable(m)
    Z = cp.bmat( [[X, np.eye(n)], [np.eye(n), U.T @ cp.diag(v) @ U]] )
    constraints = [ Z >> 0, 0 <= v, v <= 1, sum(v) == r ]
    prob = cp.Problem(cp.Minimize(cp.trace(X)), constraints)
    prob.solve(solver=cp.CVXOPT)
    return v.value

## row selection via convex relaxation

In [74]:
r = 7

In [75]:
%%time

# Apply Boyd heuristic, compute lower and upper bounds for the cost
v = BoydHeuristic(U,r)

# rounding heuristic: choose r largest values
srtmp = sorted( enumerate(v), key = lambda x : x[1], reverse=True )
select_rounding = sorted([ x[0] for x in srtmp[:r] ])

print("chosen rows = ", select_rounding)
print("achieved cost using this choice = ", costfun(U,select_rounding) )
print("lower bound on optimal cost = ", 1/n * inv( U.T @ cp.diag(v).value @ U ).trace() )

chosen rows =  [4, 22, 23, 26, 29, 53, 56]
achieved cost using this choice =  -6.7419606462169864e+16
lower bound on optimal cost =  8.110564441675473
CPU times: user 2.61 s, sys: 58.9 ms, total: 2.67 s
Wall time: 1.86 s


## row selection via brute force

In [68]:
%%time

# look at all possibilities
cs = [ (select, costfun(U,select)) for select in combinations(range(m),r) ]
cstmp = sorted( cs, key = lambda x : x[1] )[0]

print( "optmal rows = ", cstmp[0] )
print( "optimal cost = ", cstmp[1] )

KeyboardInterrupt: 

In [ ]:
# plot all possible achievable costs (all possible subsets)
cns = sorted([ x[1] for x in cs ])
plt.figure(figsize=(10,5))
plt.semilogy(cns)
plt.xlim(xmin=0,xmax=len(cns)-1)
plt.grid(which='both')
plt.xlabel(f"random subset of {r} rows out of a random {m}x{n} orthogonal matrix")
plt.ylabel("objective function, trace of pinv(U)");

In [ ]:
# approximate the distribution of costs obtained if the rows are sampled randomly without replacement
np.random.seed(1)
rx = np.random.randint(0,len(cns),size=1000000)
samples = [ cns[k] for k in rx ]

plt.figure(figsize=(10,5))
plt.hist( samples, bins=np.logspace(0,3,100), log=False, density=True )
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel(f"value obtained by sampling a random subset of {r} rows out of a random {m}x{n} orthogonal matrix")
plt.ylabel("probability density")
plt.grid(which='both')